<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaMergeAudit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaMerge") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [4]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [5]:
df.createOrReplaceTempView('Products_temp')

In [6]:
spark.sql('select * from products_temp').show(4)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 4 rows



In [7]:
spark.sql('create or replace table Products (product string,store string,product_id string,MRP int,\
            state string, country string)using delta location "/content/sample_data/store"')

DataFrame[]

In [8]:
spark.sql('select * from Products').show()

+-------------------+---------------+----------+----+----------+-------+
|            product|          store|product_id| MRP|     state|country|
+-------------------+---------------+----------+----+----------+-------+
|      Wired Earbuds|Electronics Hub|   PROD101|  25|California|    USA|
|   Portable Charger|    Mobile Mart|   PROD102|null|     Texas|    USA|
|  Bluetooth Speaker|Electronics Hub|   PROD103|  79|California|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104|  15|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105|  49|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106|null|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| 120|  New York|    USA|
|             Webcam|Electronics Hub|   PROD108|  55|      NULL|    USA|
|     Gaming Headset|      PC Palace|   PROD109|  90|  New York|    USA|
|      Monitor Stand|   Home Gadgets|   PROD110|  29|   Florida|    USA|
|         HDMI Cable|    Mobile Mart|   PROD111|nul

In [9]:
spark.sql(
    "Merge into Products as target using Products_temp as source on target.product_id = source.product_id\
     when matched then update set target.product=source.product,\
        target.store=source.store,\
        target.product_id=source.product_id,\
        target.MRP=source.MRP,\
        target.state=source.state,\
        target.country=source.country\
     when not matched then insert * "
)

DataFrame[]

In [10]:
spark.sql('select * from Products').show(5)

+-----------------+---------------+----------+----+----------+-------+
|          product|          store|product_id| MRP|     state|country|
+-----------------+---------------+----------+----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101|  25|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102|null|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103|  79|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104|  15|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105|  49|     Texas|    USA|
+-----------------+---------------+----------+----+----------+-------+
only showing top 5 rows



Audit Log

In [11]:
spark.sql("describe history Products").show(truncate=False)

+-------+-----------------------+------+--------+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation              |operationParameters                                                                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                         

In [12]:
delta_df=DeltaTable.forPath(spark,'/content/sample_data/store')

In [13]:
audit=delta_df.history()

In [14]:
audit.show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      3|2025-06-17 08:59:...|  null|    null|               MERGE|{predicate -> (ta...|null|    null|     null|          2|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.2....|
|      2|2025-06-17 08:25:...|  null|    null|               MERGE|{predicate -> (ta...|null|    null|     null|          1|  Serializable|        false|{numTargetR

In [15]:
audit.show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      3|2025-06-17 08:59:...|  null|    null|               MERGE|{predicate -> (ta...|null|    null|     null|          2|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.2....|
|      2|2025-06-17 08:25:...|  null|    null|               MERGE|{predicate -> (ta...|null|    null|     null|          1|  Serializable|        false|{numTargetR

In [18]:
audit_log=audit.select(audit.operation,audit.version,explode(audit.operationMetrics).alias('key','value'))
audit_log.show(5)

+---------+-------+--------------------+-----+
|operation|version|                 key|value|
+---------+-------+--------------------+-----+
|    MERGE|      3| numTargetRowsCopied|    0|
|    MERGE|      3|numTargetRowsDeleted|    0|
|    MERGE|      3| numTargetFilesAdded|    1|
|    MERGE|      3|     executionTimeMs| 4225|
|    MERGE|      3|numTargetRowsInse...|    0|
+---------+-------+--------------------+-----+
only showing top 5 rows



In [17]:
audit_log_table=audit_log.groupBy('operation','version').pivot('key').agg(first("value"))
audit_log_table.show()

+---------+-------+---------------+-------------+-------------+-------------------------+-------------------+---------------------+-------------------+--------------------+---------------------+--------------------+-------------+----------+
|operation|version|executionTimeMs|numOutputRows|numSourceRows|numTargetChangeFilesAdded|numTargetFilesAdded|numTargetFilesRemoved|numTargetRowsCopied|numTargetRowsDeleted|numTargetRowsInserted|numTargetRowsUpdated|rewriteTimeMs|scanTimeMs|
+---------+-------+---------------+-------------+-------------+-------------------------+-------------------+---------------------+-------------------+--------------------+---------------------+--------------------+-------------+----------+
|    MERGE|      1|           5610|          100|          100|                        0|                  1|                    0|                  0|                   0|                  100|                   0|         2500|      3092|
|    MERGE|      2|           5186| 

**END OF CODE**